In [3]:
import pickle
import pandas as pd
import math
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor

# Load data

In [20]:
df_touse, df_predict = pickle.load(open('datasets.sav','rb'))

In [21]:
df = df_touse

## Splitting dataset and storing descriptive statsitics and indices of splits to process whole dataframe to prevent data leakage

In [22]:
predictors = df.columns.tolist()
predictors.remove("target")
X = df[predictors]
y = df["target"]


X_train, X_test, Y_train, Y_test = train_test_split(X,y,test_size=0.2)

In [49]:
test_index_values = X_test.index.values
train_index_values = Y_train.index.values


# Select only the numerical columns
numeric_cols = X_train.select_dtypes(include=['int', 'float'])

# Compute the mean of each numerical column
X_train_mean = numeric_cols.mean()
X_test_mean = numeric_cols.mean()




# Compute the mean of each numerical column
Y_train_mean = Y_train.mean()




# Compute the mean of each numerical column
Y_test_mean = Y_test.mean()


# Transform data
## numerical NaNs replaced by Mean of respective split

In [57]:
df = df.drop(columns=['insee'])

In [58]:
df.loc[train_index_values, ['PERSON_ID', 'AGE_2018', 'PAY', 'Working_hours']] = df.loc[train_index_values, ['PERSON_ID', 'AGE_2018', 'PAY', 'Working_hours']].fillna(X_train_mean)

df.loc[test_index_values, ['PERSON_ID', 'AGE_2018', 'PAY', 'Working_hours']] = df.loc[test_index_values, ['PERSON_ID', 'AGE_2018', 'PAY', 'Working_hours']].fillna(X_test_mean)

df.loc[train_index_values, ['target']] = df.loc[train_index_values, ['target']].fillna(Y_train_mean)

df.loc[test_index_values, ['target']] = df.loc[test_index_values, ['target']].fillna(Y_test_mean)

## categorical NaNs replaced by random category of variable while respecting proportions of respective split

In [64]:

# Select only the categorical columns
cat_cols = df.select_dtypes(include=['object'])

## For the training set

# Iterate over the categorical columns
for col_name in cat_cols.columns:
    # Select the column
    col = X_train[col_name]
    
    # Compute the proportions of the categories
    counts = col.value_counts(normalize=True)
    
    # Replace the NaN values with random categories
    for i in col.index:
        if pd.isnull(col[i]):
            col[i] = random.choices(counts.index, counts.values)[0]
            
df.update(col)


## For the test set

for col_name in cat_cols.columns:
    # Select the column
    col = X_test[col_name]
    
    # Compute the proportions of the categories
    counts = col.value_counts(normalize=True)
    
    # Replace the NaN values with random categories
    for i in col.index:
        if pd.isnull(col[i]):
            col[i] = random.choices(counts.index, counts.values)[0]
            
df.update(col)



/var/folders/l6/36kbn2fd34q7z3t8k7r87g8h0000gn/T/ipykernel_31276/3862709179.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  col[i] = random.choices(counts.index, counts.values)[0]
/var/folders/l6/36kbn2fd34q7z3t8k7r87g8h0000gn/T/ipykernel_31276/3862709179.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  col[i] = random.choices(counts.index, counts.values)[0]
/var/folders/l6/36kbn2fd34q7z3t8k7r87g8h0000gn/T/ipykernel_31276/3862709179.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

## One Hot Encode categorical variables

In [66]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
## For the Train set

# Select the string columns
string_columns = df.select_dtypes(['object']).columns

# Create a OneHotEncoder object
encoder = OneHotEncoder()

# Fit the encoder to the string columns
encoder.fit(df[string_columns])

# Encode the string columns
encoded_data = encoder.transform(df[string_columns])

# Create a DataFrame with the encoded columns and the original index
encoded_df = pd.DataFrame(encoded_data.toarray(), index=df.index, columns=encoder.get_feature_names(string_columns))

# Concatenate the encoded columns with the rest of the data
df = pd.concat([encoded_df, df.drop(string_columns, axis=1)], axis=1)

/Users/jan-philippkretschmann/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## Reassign processed splits 

In [79]:
X_train = df.loc[train_index_values]
X_train = X_train.drop('target', axis=1)
X_test =  df.loc[test_index_values]
X_test = X_test.drop('target', axis=1)
Y_train = df.loc[train_index_values]['target']
Y_test = df.loc[test_index_values]['target']

# Models

## DecisionTree Regression

In [83]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import StratifiedKFold

dt_params = {'min_samples_split': [2, 5] + list(range(10, 100, 5))}

#,
#              'max_features': ['auto', 'sqrt', 'log2', None],
#              'min_impurity_decrease': [0.0, 0.1, 0.2, 0.3]}


#dt_params = {'min_samples_split': [2, 5] + list(range(10, 250,5))} 
dt = DecisionTreeRegressor(random_state=0)
cv_folds = KFold(5, shuffle=True, random_state=0)
dt_cv = GridSearchCV(dt, dt_params, cv=cv_folds, n_jobs=-1) 
dt_cv.fit(X_train, Y_train) 
print(dt_cv.best_score_)
print(dt_cv.best_params_)

0.5642934130439852
{'min_samples_split': 95}


In [85]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import math

# Make predictions on the test data
predictions_dt = dt_cv.predict(X_test)

# Compute the mean absolute error
mae = mean_absolute_error(Y_test, predictions_dt)

# Compute the mean squared error
mse = mean_squared_error(Y_test, predicpredictions_dttions)

# Compute the root mean squared error
rmse = math.sqrt(mse)

# Compute the R-squared score
r2 = r2_score(Y_test, predictions_dt)

# Compute the adjusted R-squared score
n = len(Y_test)
p = X_test.shape[1]  # number of features
adj_r2 = 1 - (1-r2)*(n-1)/(n-p-1)



print("Mean absolute error: {:.2f}".format(mae))
print("Mean squared error: {:.2f}".format(mse))
print("Root mean squared error: {:.2f}".format(rmse))
print("R-squared score: {:.2f}".format(r2))
print("Adjusted R-squared score: {:.2f}".format(adj_r2))



Mean absolute error: 6.49
Mean squared error: 71.88
Root mean squared error: 8.48
R-squared score: 0.58
Adjusted R-squared score: 0.54


## GradientBoosting Regression

In [86]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, GridSearchCV

# Define the grid of hyperparameters to search
param_grid = {'learning_rate': [0.2,0.1, 0.05, 0.01, 0.001]}


# Create the gradient boosting model
gb = GradientBoostingRegressor(random_state=0)

# Create the K-fold cross-validation object
cv_folds = KFold(5, shuffle=True, random_state=0)

# Create the grid search object
gb_cv = GridSearchCV(gb, param_grid, cv=cv_folds, n_jobs=-1)

# Fit the grid search object to the training data
gb_cv.fit(X_train, Y_train)

print(gb_cv.best_score_)
print(gb_cv.best_params_)

0.6957355354397229
{'learning_rate': 0.2}


In [87]:
# Make predictions on the test data
predictions_gb = gb_cv.predict(X_test)

# Compute the mean absolute error
mae = mean_absolute_error(Y_test, predictions_gb)

# Compute the mean squared error
mse = mean_squared_error(Y_test, predictions_gb)

# Compute the root mean squared error
rmse = math.sqrt(mse)

# Compute the R-squared score
r2 = r2_score(Y_test, predictions_gb)

# Compute the adjusted R-squared score
n = len(Y_test)
p = X_test.shape[1]  # number of features
adj_r2 = 1 - (1-r2)*(n-1)/(n-p-1)



print("Mean absolute error: {:.2f}".format(mae))
print("Mean squared error: {:.2f}".format(mse))
print("Root mean squared error: {:.2f}".format(rmse))
print("R-squared score: {:.2f}".format(r2))
print("Adjusted R-squared score: {:.2f}".format(adj_r2))


Mean absolute error: 5.55
Mean squared error: 49.87
Root mean squared error: 7.06
R-squared score: 0.71
Adjusted R-squared score: 0.68


## Ridge model

In [94]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold, GridSearchCV

# Define the grid of hyperparameters to search
param_grid = {'alpha': [0.1, 1.0, 10.0, 15.0, 100.0, 1000.0]}
              #,'max_iter': [100, 1000, 10000, 100000]}

# Create the ridge regression model
ridge = Ridge(random_state=0)

# Create the K-fold cross-validation object
cv_folds = KFold(5, shuffle=True, random_state=0)

# Create the grid search object
ridge_cv = GridSearchCV(ridge, param_grid, cv=cv_folds, n_jobs=-1)

# Fit the grid search object to the training data
ridge_cv.fit(X_train, Y_train)
print(ridge_cv.best_score_)
print(ridge_cv.best_params_)

0.6123357258856378
{'alpha': 15.0}


In [89]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import math

# Make predictions on the test data
predictions_ridge = ridge_cv.predict(X_test)

# Compute the mean absolute error
mae = mean_absolute_error(Y_test, predictions_ridge)

# Compute the mean squared error
mse = mean_squared_error(Y_test, predictions_ridge)

# Compute the root mean squared error
rmse = math.sqrt(mse)

# Compute the R-squared score
r2 = r2_score(Y_test, predictions_ridge)

# Compute the adjusted R-squared score
n = len(Y_test)
p = X_test.shape[1]  # number of features
adj_r2 = 1 - (1-r2)*(n-1)/(n-p-1)



print("Mean absolute error: {:.2f}".format(mae))
print("Mean squared error: {:.2f}".format(mse))
print("Root mean squared error: {:.2f}".format(rmse))
print("R-squared score: {:.2f}".format(r2))
print("Adjusted R-squared score: {:.2f}".format(adj_r2))



Mean absolute error: 6.39
Mean squared error: 64.56
Root mean squared error: 8.04
R-squared score: 0.62
Adjusted R-squared score: 0.59


## RandomForest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, GridSearchCV

# Define the grid of hyperparameters to search
param_grid = {'n_estimators': [5,7,10]}
              #'max_depth': [2, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
              #'min_samples_split': [2, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
              #'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}

# Create the random forest model
rf = RandomForestRegressor(random_state=0)

# Create the K-fold cross-validation object
cv_folds = KFold(5, shuffle=True, random_state=0)

# Create the grid search object
rf_cv = GridSearchCV(rf, param_grid, cv=cv_folds, n_jobs=-1)

# Fit the grid search object to the training data
rf_cv.fit(X_train, Y_train)
print(rf_cv.best_score_)
print(rf_cv.best_params_)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import math

# Make predictions on the test data
predictions = rf_cv.predict(X_test)

# Compute the mean absolute error
mae = mean_absolute_error(Y_test, predictions)

# Compute the mean squared error
mse = mean_squared_error(Y_test, predictions)

# Compute the root mean squared error
rmse = math.sqrt(mse)

# Compute the R-squared score
r2 = r2_score(Y_test, predictions)

# Compute the adjusted R-squared score
n = len(Y_test)
p = X_test.shape[1]  # number of features
adj_r2 = 1 - (1-r2)*(n-1)/(n-p-1)



print("Mean absolute error: {:.2f}".format(mae))
print("Mean squared error: {:.2f}".format(mse))
print("Root mean squared error: {:.2f}".format(rmse))
print("R-squared score: {:.2f}".format(r2))
print("Adjusted R-squared score: {:.2f}".format(adj_r2))

## LASSO

In [98]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold, GridSearchCV

# Define the grid of hyperparameters to search
param_grid = {'alpha': [0.005, 0.001, 0.00005]}
              #,'max_iter': [100, 1000, 10000, 100000]}

# Create the Lasso regression model
lasso = Lasso(random_state=0)

# Create the K-fold cross-validation object
cv_folds = KFold(5, shuffle=True, random_state=0)

# Create the grid search object
lasso_cv = GridSearchCV(lasso, param_grid, cv=cv_folds, n_jobs=-1)

# Fit the grid search object to the training data
lasso_cv.fit(X_train, Y_train)
print(lasso_cv.best_score_)
print(lasso_cv.best_params_)

/Users/jan-philippkretschmann/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.748e+05, tolerance: 5.430e+02
  model = cd_fast.enet_coordinate_descent(
/Users/jan-philippkretschmann/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.297e+04, tolerance: 5.499e+02
  model = cd_fast.enet_coordinate_descent(
/Users/jan-philippkretschmann/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale 

0.6125471487834089
{'alpha': 0.001}


/Users/jan-philippkretschmann/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.272e+05, tolerance: 6.827e+02
  model = cd_fast.enet_coordinate_descent(


In [100]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import math

# Make predictions on the test data
predictions_lasso = lasso_cv.predict(X_test)

# Compute the mean absolute error
mae = mean_absolute_error(Y_test, predictions_lasso)

# Compute the mean squared error
mse = mean_squared_error(Y_test, predictions_lasso)

# Compute the root mean squared error
rmse = math.sqrt(mse)

# Compute the R-squared score
r2 = r2_score(Y_test, predictions_lasso)

# Compute the adjusted R-squared score
n = len(Y_test)
p = X_test.shape[1]  # number of features
adj_r2 = 1 - (1-r2)*(n-1)/(n-p-1)



print("Mean absolute error: {:.2f}".format(mae))
print("Mean squared error: {:.2f}".format(mse))
print("Root mean squared error: {:.2f}".format(rmse))
print("R-squared score: {:.2f}".format(r2))
print("Adjusted R-squared score: {:.2f}".format(adj_r2))

Mean absolute error: 6.40
Mean squared error: 64.55
Root mean squared error: 8.03
R-squared score: 0.62
Adjusted R-squared score: 0.59
